In [1]:
basePath = '/home/ps/yulj/PycharmProjects/pythonProject/Wav2Lip-GFPGAN'
audio_file = basePath + '/inputs/1.wav'

face_file = basePath + '/outputs/result.mp4'


wav2lipFolderName = 'Wav2Lip-master'
wav2lipPath = basePath + '/' + wav2lipFolderName

%cd $wav2lipPath

/home/ps/yulj/PycharmProjects/pythonProject/Wav2Lip-GFPGAN/Wav2Lip-master


In [2]:
import audio
import numpy as np
import cv2
import librosa
import time

print(f'import ok')

import ok


In [ ]:
import argparse
import asyncio
import json
import logging
import os
import ssl
import uuid

import cv2
from aiohttp import web
from aiortc import MediaStreamTrack, RTCPeerConnection, RTCSessionDescription
from aiortc.contrib.media import MediaBlackhole, MediaPlayer, MediaRecorder, MediaRelay
from av import VideoFrame, AudioFrame
import time
import queue
import numpy as np
import fractions
import nest_asyncio
from typing import Tuple, Union
from aiortc.mediastreams import AUDIO_PTIME, VIDEO_CLOCK_RATE, VIDEO_PTIME, VIDEO_TIME_BASE, MediaStreamError


In [ ]:
player = MediaPlayer(file = '/home/ps/yulj/PycharmProjects/pythonProject/source/aiortc/examples/server/5a7bdba7-563a-44ce-95fa-ded35217310b1_2.wav')

In [5]:
video_stream = cv2.VideoCapture("/home/ps/yulj/PycharmProjects/pythonProject/Wav2Lip-GFPGAN/Wav2Lip-master/results/trim_ipseek_copy_result.mp4")
fps = video_stream.get(cv2.CAP_PROP_FPS)
print(fps)

25.0


In [29]:
video_stream = cv2.VideoCapture(face_file)
#video_stream = cv2.VideoCapture("/home/ai/yulj/PycharmProjects/pythonProject/Wav2Lip-GFPGAN/Wav2Lip-master/results/trim_ipseek_copy_result.mp4")
fps = video_stream.get(cv2.CAP_PROP_FPS)

print('Reading video frames...', fps)

full_frames = []
while 1:
	still_reading, frame = video_stream.read()
	if not still_reading:
		video_stream.release()
		break


	y1, y2, x1, x2 = [0, -1, 0, -1]
	if x2 == -1: x2 = frame.shape[1]
	if y2 == -1: y2 = frame.shape[0]

	frame = frame[y1:y2, x1:x2]

	full_frames.append(frame)
	break

print ("Number of frames available for inference: "+str(len(full_frames)))

Reading video frames... 59.94005994005994
Number of frames available for inference: 1


In [32]:
print(full_frames[0])
print('ffff')
print(np.asarray(full_frames)[0])

[[[229 221 216]
  [229 221 216]
  [229 221 216]
  ...
  [245 248 246]
  [245 248 246]
  [245 248 246]]

 [[229 221 216]
  [229 221 216]
  [229 221 216]
  ...
  [245 248 246]
  [245 248 246]
  [245 248 246]]

 [[229 221 216]
  [229 221 216]
  [229 221 216]
  ...
  [245 248 246]
  [245 248 246]
  [245 248 246]]

 ...

 [[178 181 179]
  [161 164 162]
  [183 186 184]
  ...
  [203 207 209]
  [209 213 215]
  [197 201 203]]

 [[164 167 165]
  [168 171 169]
  [187 190 188]
  ...
  [209 213 215]
  [206 210 212]
  [201 205 207]]

 [[169 172 170]
  [173 176 174]
  [175 178 176]
  ...
  [211 215 217]
  [206 210 212]
  [206 210 212]]]
ffff
[[[229 221 216]
  [229 221 216]
  [229 221 216]
  ...
  [245 248 246]
  [245 248 246]
  [245 248 246]]

 [[229 221 216]
  [229 221 216]
  [229 221 216]
  ...
  [245 248 246]
  [245 248 246]
  [245 248 246]]

 [[229 221 216]
  [229 221 216]
  [229 221 216]
  ...
  [245 248 246]
  [245 248 246]
  [245 248 246]]

 ...

 [[178 181 179]
  [161 164 162]
  [183 186 184]

In [30]:
print(full_frames[0].shape)
after_frame = cv2.resize(full_frames[0], (full_frames[0].shape[1]//4, full_frames[0].shape[0]//4))
print(after_frame.shape)

(1920, 1080, 3)
(480, 270, 3)


In [37]:
print(full_frames[1][0])

[[232 222 217]
 [232 222 217]
 [232 222 217]
 ...
 [248 248 246]
 [248 248 246]
 [248 248 246]]


In [33]:
full_frames[0].shape[:-1]

(1920, 1080)

In [36]:
audio_file = '/home/ps/yulj/PycharmProjects/pythonProject/Wav2Lip-GFPGAN/inputs/zhiqi.wav'
print(f'time now: {int(time.time())}')
wav = audio.load_wav(audio_file, 16000)
mel = audio.melspectrogram(wav)
print(f'time now: {int(time.time())}')
print(wav)
print(np.shape(wav))
print(wav.shape[0]/ 16000.0)
print(mel.shape)
audio.save_wav(wav, '/home/ps/yulj/PycharmProjects/pythonProject/Wav2Lip-GFPGAN/inputs/zhiqi_2.wav', 16000)
audio.save_wavenet_wav(wav, '/home/ps/yulj/PycharmProjects/pythonProject/Wav2Lip-GFPGAN/inputs/zhiqi_3.wav', 16000)

time now: 1684118253
time now: 1684118253
[0.00049419 0.00053025 0.00017894 ... 0.         0.         0.        ]
(53600,)
3.35
(80, 269)


In [24]:
mel_chunks = []

fps = 25
mel_idx_multiplier = 80. / fps
i = 0
mel_step_size = 16
while 1:
    start_idx = int(i * mel_idx_multiplier)
    if start_idx + mel_step_size > len(mel[0]):
        mel_chunks.append(mel[:, len(mel[0]) - mel_step_size:])
        break
    mel_chunks.append(mel[:, start_idx: start_idx + mel_step_size])
    i += 1

print("Length of mel chunks: {}".format(len(mel_chunks)))

print(f'shape of mel chunks: ', np.shape(mel_chunks))
print(f'len of mel chunk 0: ', len(mel_chunks[0]))

print(f'shape of mel: ', np.shape(mel))
print(f'len of mel 0: ', len(mel[0]))

Length of mel chunks: 81
shape of mel chunks:  (81, 80, 16)
len of mel chunk 0:  80
shape of mel:  (80, 269)
len of mel 0:  269


In [27]:
import numpy as np
import librosa

# 读取音频文件
y, sr = librosa.load(audio_file)

# 计算梅尔频谱
mel_spec = librosa.feature.melspectrogram(S=librosa.stft(y), sr=sr)

#mel_spec = mel
# 将梅尔频谱转换为对数尺度下的功率谱
log_mel_spec = librosa.amplitude_to_db(mel_spec, ref=np.max)

# 重构音频信号
audio = librosa.istft(log_mel_spec, hop_length=256)

# 保存音频文件
audio = audio * 32767 # 转换为16位整型
librosa.output.write_wav('/home/ps/yulj/PycharmProjects/pythonProject/Wav2Lip-GFPGAN/inputs/audio_file_out.wav', audio, sr)

/home/ps/anaconda3/envs/wav2lip/lib/python3.6/site-packages/librosa/core/spectrum.py:1029: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  warnings.warn('amplitude_to_db was called on complex input so phase '


In [10]:
d = librosa.get_duration(y=wav, sr=16000, S=None, n_fft=2048, hop_length=512, center=True, filename=None)
print(d)


7.8


In [28]:
sr = librosa.get_samplerate(audio_file)
sr

48000

In [ ]:
import librosa.display
import matplotlib.pyplot as plt
y, sr = librosa.load(audio_file, duration=10)
librosa.display.waveplot(y, sr=sr)
plt.show()

In [22]:

mel = audio.melspectrogram(wav)
print(mel.shape)
print(mel)
print(np.shape(mel))

(80, 625)
[[-4.        -4.        -4.        ... -2.7683058 -2.5265422 -2.5346613]
 [-4.        -4.        -4.        ... -1.7465076 -1.8171735 -1.9555194]
 [-4.        -4.        -4.        ... -1.157727  -1.395241  -1.2758422]
 ...
 [-4.        -4.        -4.        ... -4.        -4.        -3.9292579]
 [-4.        -4.        -4.        ... -4.        -4.        -3.6217425]
 [-4.        -4.        -4.        ... -4.        -4.        -4.       ]]
(80, 625)


In [6]:
print(mel.reshape(-1))

[-4. -4. -4. ... -4. -4. -4.]


In [7]:
80./60


1.3333333333333333

In [5]:
124800/16000


7.8

AttributeError: 'list' object has no attribute 'shape'

In [6]:
a = np.mat([[1, 2, 3], [4, 5, 6],[4, 5, 6],[4, 5, 6],[4, 5, 6],[4, 5, 6],[4, 5, 6]])

In [7]:
print(a.shape)

(7, 3)


In [19]:
a[:,0:2:]

matrix([[1, 2],
        [4, 5],
        [4, 5],
        [4, 5],
        [4, 5],
        [4, 5],
        [4, 5]])

In [17]:
mel_idx_multiplier = 80./60
for i in range(0, 10):
    print(i, int(i * mel_idx_multiplier))

0 0
1 1
2 2
3 4
4 5
5 6
6 8
7 9
8 10
9 12


In [25]:
mel_chunks = []
mel_step_size = 16
mel_idx_multiplier = 80./fps
i = 0
while 1:
	start_idx = int(i * mel_idx_multiplier)
	if start_idx + mel_step_size > len(mel[0]):
		mel_chunks.append(mel[:, len(mel[0]) - mel_step_size:])
		break
	mel_chunks.append(mel[:, start_idx : start_idx + mel_step_size])
	i += 1

print("Length of mel chunks: {}".format(len(mel_chunks)))

Length of mel chunks: 459


In [29]:
for i, m in enumerate(mel_chunks):
	print(i, m)

0 [[-4.         -4.         -4.         ... -1.869911   -3.154298
  -3.8193011 ]
 [-4.         -4.         -4.         ... -1.8173668  -3.1289282
  -3.6709876 ]
 [-4.         -4.         -4.         ... -1.7851195  -3.2710333
  -3.6735034 ]
 ...
 [-4.         -4.         -4.         ... -1.1473076  -1.0817275
  -0.74990654]
 [-4.         -4.         -4.         ... -1.4763403  -1.2046936
  -0.95941734]
 [-4.         -4.         -4.         ... -2.0335913  -1.647788
  -1.3237219 ]]
1 [[-4.         -4.         -4.         ... -3.154298   -3.8193011
  -4.        ]
 [-4.         -4.         -4.         ... -3.1289282  -3.6709876
  -3.6869166 ]
 [-4.         -4.         -4.         ... -3.2710333  -3.6735034
  -3.5723119 ]
 ...
 [-4.         -4.         -4.         ... -1.0817275  -0.74990654
  -0.59106207]
 [-4.         -4.         -4.         ... -1.2046936  -0.95941734
  -0.7451117 ]
 [-4.         -4.         -4.         ... -1.647788   -1.3237219
  -1.1535292 ]]
2 [[-4.         -4.     

In [32]:
from tqdm import tqdm
for x,i in enumerate(tqdm(range(1, 10, 2))):
	print(x,i)



100%|██████████| 5/5 [00:00<00:00, 2633.62it/s]

0 1
1 3
2 5
3 7
4 9


In [3]:
a=[1,2,3]
a

[1, 2, 3]

In [5]:
np.shape(a)

(3,)

In [6]:
a=[[1,2,3]]
np.shape(a)

(1, 3)

In [7]:
np.shape(a)

(1, 3)

In [10]:
np.reshape(a, '1' ,'3')

ValueError: order must be one of 'C', 'F', 'A', or 'K' (got '3')

In [11]:
a.reshape(1,3)

AttributeError: 'list' object has no attribute 'reshape'

In [12]:
a_1=np.arange(1,5
			)

In [13]:
a_1


array([1, 2, 3, 4])

In [14]:
a_1.reshape(1,4)

array([[1, 2, 3, 4]])

In [15]:
a_1.len


AttributeError: 'numpy.ndarray' object has no attribute 'len'

In [16]:
len(a_1)

4

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 6)

In [19]:
import asyncio

In [25]:
async def test1():
    lit1=[]
    for i in range(4):
        await asyncio.sleep(1)
        # print(i,"test1")
        lit1.append(i)
    return lit1



In [40]:
async def ss(wait):
	await asyncio.sleep(wait)

In [35]:
import  nest_asyncio

In [36]:
nest_asyncio.apply()

In [37]:
a=asyncio.run(test1())
print(f"result: {a}")

result: [0, 1, 2, 3]


In [38]:
loop = asyncio.get_event_loop()
            # blackHole 一直会while 消费 track
            #result = loop.run_until_complete(start_blackhole(blackHole))

In [39]:
future1 = asyncio.ensure_future(test1())
result = loop.run_until_complete(future1)
print(result)

print(future1.result())

[0, 1, 2, 3]
[0, 1, 2, 3]


In [44]:
a=asyncio.run(ss(0.0001))
print(f"result: {a}")

result: None
